# Packages Installs, imports, and presets

In [126]:
#from scapy.all import *
import pandas as pd
#import seaborn as sns
import numpy as np
from collections import defaultdict
import os

In [127]:
INPUT_SIZE = (256,256,3)
TIMEOUT = 15

In [128]:
df = pd.read_parquet(f"data/iec104v2_{TIMEOUT}.parquet")

In [129]:
df.dtypes.value_counts()

int64     22
object     8
Name: count, dtype: int64

In [130]:
df.loc[:,df.dtypes=='object']

ip_flags  \
Flow                                               PacketNumber            
[1, '192.168.1.20', '192.168.1.21', 2404, 50045... 0                  DF   
                                                   1                  DF   
                                                   2                  DF   
                                                   3                  DF   
                                                   4                  DF   
...                                                                  ...   
[1, '192.168.1.22', '192.168.1.29', 2404, 39973... 4                  DF   
                                                   5                  DF   
                                                   6                  DF   
                                                   7                  DF   
                                                   8                  DF   

                                                                       ip_src  \
Flow                                               PacketNumber                 
[1, '192.168.1.20', '192.168.1.21', 2404, 50045... 0             192.168.1.20   
                                                   1             192.168.1.21   
                                                   2             192.168.1.21   
                                                   3             192.168.1.21   
                                                   4             192.168.1.21   
...                                                                       ...   
[1, '192.168.1.22', '192.168.1.29', 2404, 39973... 4             192.168.1.22   
                                                   5             192.168.1.22   
                                                   6             192.168.1.29   
                                                   7             192.168.1.22   
                                                   8             192.168.1.22   

                                                                       ip_dst  \
Flow                                               PacketNumber                 
[1, '192.168.1.20', '192.168.1.21', 2404, 50045... 0             192.168.1.21   
                                                   1             192.168.1.20   
                                                   2             192.168.1.20   
                                                   3             192.168.1.20   
                                                   4             192.168.1.20   
...                                                                       ...   
[1, '192.168.1.22', '192.168.1.29', 2404, 39973... 4             192.168.1.29   
                                                   5             192.168.1.29   
                                                   6             192.168.1.22   
                                                   7             192.168.1.29   
                                                   8             192.168.1.29   

                                                                              time  \
Flow                                               PacketNumber                      
[1, '192.168.1.20', '192.168.1.21', 2404, 50045... 0             1587763927.678300   
                                                   1             1587763927.680222   
                                                   2             1587763927.880137   
                                                   3             1587763928.778027   
                                                   4             1587763928.989181   
...                                                                            ...   
[1, '192.168.1.22', '192.168.1.29', 2404, 39973... 4             1588172481.916780   
                                                   5             1588172481.917103   
                                                   6             1588172491.959546   
                     

In [131]:
df['time']=df['time'].astype(float)

In [132]:
df.loc[:,df.dtypes=='int']

ip_version  \
Flow                                               PacketNumber               
[1, '192.168.1.20', '192.168.1.21', 2404, 50045... 0                      4   
                                                   1                      4   
                                                   2                      4   
                                                   3                      4   
                                                   4                      4   
...                                                                     ...   
[1, '192.168.1.22', '192.168.1.29', 2404, 39973... 4                      4   
                                                   5                      4   
                                                   6                      4   
                                                   7                      4   
                                                   8                      4   

                                                                 ip_ihl  \
Flow                                               PacketNumber           
[1, '192.168.1.20', '192.168.1.21', 2404, 50045... 0                  5   
                                                   1                  5   
                                                   2                  5   
                                                   3                  5   
                                                   4                  5   
...                                                                 ...   
[1, '192.168.1.22', '192.168.1.29', 2404, 39973... 4                  5   
                                                   5                  5   
                                                   6                  5   
                                                   7                  5   
                                                   8                  5   

                                                                 ip_tos  \
Flow                                               PacketNumber           
[1, '192.168.1.20', '192.168.1.21', 2404, 50045... 0                  0   
                                                   1                  0   
                                                   2                  0   
                                                   3                  0   
                                                   4                  0   
...                                                                 ...   
[1, '192.168.1.22', '192.168.1.29', 2404, 39973... 4                  0   
                                                   5                  0   
                                                   6                  0   
                                                   7                  0   
                                                   8                  0   

                                                                 ip_len  \
Flow                                               PacketNumber           
[1, '192.168.1.20', '192.168.1.21', 2404, 50045... 0                 46   
                                                   1                 46   
                                                   2                 88   
                                                   3                 56   
                                                   4                 72   
...                                                                 ...   
[1, '192.168.1.22', '192.168.1.29', 2404, 39973... 4                 68   
                                                   5                 68   
                                                   6                 58   
                                                   7                 58   
                                                   8                 58   

                                                                 ip_id  \
Flow                               

# Model training

## Dataset preparation

In [133]:
df[['label']].value_counts()

label        
normal           219068
m_sp_na_1_DoS    181249
c_rd_na_1_DoS     20167
c_rp_na_1_DoS     19949
c_sc_na_1_DoS     16615
c_rp_na_1         11517
c_se_na_1_DoS     10854
c_rd_na_1         10821
c_ci_na_1_DoS      9146
c_se_na_1          6960
c_sc_na_1          6767
c_ci_na_1          5264
Name: count, dtype: int64

In [134]:
df[['label']]

label
Flow                                               PacketNumber               
[1, '192.168.1.20', '192.168.1.21', 2404, 50045... 0             m_sp_na_1_DoS
                                                   1             m_sp_na_1_DoS
                                                   2             m_sp_na_1_DoS
                                                   3             m_sp_na_1_DoS
                                                   4             m_sp_na_1_DoS
...                                                                        ...
[1, '192.168.1.22', '192.168.1.29', 2404, 39973... 4             c_sc_na_1_DoS
                                                   5             c_sc_na_1_DoS
                                                   6             c_sc_na_1_DoS
                                                   7             c_sc_na_1_DoS
                                                   8             c_sc_na_1_DoS

[518377 rows x 1 columns]

In [137]:
from feature_engine.encoding import OrdinalEncoder
od = OrdinalEncoder(encoding_method='arbitrary')
od.fit(df[['label']])
df[['label']]= od.transform(df[['label']])

In [140]:
tmp = pd.DataFrame(df['label'].unique().reshape(-1, 1),columns=['label'])
pd.concat((od.inverse_transform(tmp),tmp),axis=1)

,label,label
0,m_sp_na_1_DoS,0
1,normal,1
2,c_ci_na_1,2
3,c_se_na_1,3
4,c_sc_na_1,4
5,c_rd_na_1,5
6,c_rp_na_1,6
7,c_ci_na_1_DoS,7
8,c_rd_na_1_DoS,8
9,c_rp_na_1_DoS,9


In [141]:
TRAIN_RATE= 0.8
import random
indexes = list(set(df.index))
random.Random(1).shuffle(indexes)
TRAIN_SIZE = int(len(indexes)*TRAIN_RATE)
TEST_SIZE = len(indexes)-TRAIN_SIZE
TRAIN_SAMPLES = indexes[:TRAIN_SIZE]
TEST_SAMPLES = indexes[:TEST_SIZE]
df_train_initial = df.loc[TRAIN_SAMPLES]
df_test = df.loc[TEST_SAMPLES]
# df_test = df.iloc[TRAIN_SAMPLES:]
# del df

In [142]:
indexes = list(set(df_train_initial.index))
random.Random(1).shuffle(indexes)
TRAIN_SIZE = int(len(indexes)*TRAIN_RATE)
TEST_SIZE = len(indexes)-TRAIN_SIZE
TRAIN_SAMPLES = indexes[:TRAIN_SIZE]
TEST_SAMPLES = indexes[:TEST_SIZE]
df_train = df_train_initial.loc[TRAIN_SAMPLES]
df_validation = df_train_initial.loc[TEST_SAMPLES]

In [143]:
indexes[:3]

[("[1, '192.168.1.19', '192.168.1.29', 2404, 39543, 'TCP', 'c_rd_na_1_DoS']",
  3),
 ("[1, '192.168.1.20', '192.168.1.24', 2404, 50046, 'TCP', 'm_sp_na_1_DoS']",
  10514),
 ("[1, '192.168.1.20', '192.168.1.25', 2404, 50047, 'TCP', 'm_sp_na_1_DoS']",
  6845)]

In [144]:
df_train.shape, df_validation.shape, df_test.shape

((331760, 30), (82941, 30), (103676, 30))

In [145]:
len(set(df_train.index))

331760

In [146]:
display(df_train['label'].value_counts())
display(df_validation['label'].value_counts())
display(df_test['label'].value_counts())

label
1     139940
0     116121
8      12884
9      12749
11     10627
6       7322
5       7031
10      6969
7       5809
3       4479
4       4410
2       3419
Name: count, dtype: int64

label
1     34792
0     29075
9      3299
8      3209
11     2670
6      1834
5      1735
10     1704
7      1456
4      1155
3      1152
2       860
Name: count, dtype: int64

label
1     43705
0     36294
8      4039
9      3966
11     3338
6      2270
5      2207
10     2157
7      1858
4      1452
3      1360
2      1030
Name: count, dtype: int64

In [148]:
df_train.to_parquet(f'data/iec104_train_{TIMEOUT}_v2.parquet')
df_validation.to_parquet(f'data/iec104_validation_{TIMEOUT}_v2.parquet')
df_test.to_parquet(f'data/iec104_test_{TIMEOUT}_v2.parquet')